# Create LinkedIn Post
Creates an engaging LinkedIn post on topics pertinent to my profile

## Setup

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-community langchain-openai composio-langgraph duckduckgo-search langchain-unstructured beautifulsoup4 unstructured playwright

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [3]:
_set_env("LANGCHAIN_API_KEY")
_set_env("LANGCHAIN_TRACING_V2")
_set_env("LANGCHAIN_PROJECT")
_set_env("LANGCHAIN_ENDPOINT")

In [4]:
_set_env("OPENAI_API_KEY")

In [5]:
_set_env("COMPOSIO_API_KEY")

## Tools

In [6]:
tools = []

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

tools.append(search)

In [8]:
from typing import List

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool

@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
# tools.append(scrape_webpages)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# from typing import Literal
# from composio_langgraph import ComposioToolSet, App

# composio_toolset = ComposioToolSet()
# tools = composio_toolset.get_tools(
#     apps=[App.GITHUB]
# )


In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

## Agent

In [ ]:
from langchain_openai import ChatOpenAI

model = None
if os.getenv("OPENAI_API_KEY", None):
    model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=os.getenv("TEMPERATURE", 0),
        streaming=os.getenv("STREAMING", True)
    )
else:
    raise Exception("Cannot infer chat model from environment variables")


In [ ]:
from langchain_core.messages import AIMessage

async def agent_node(state, agent, name):
    """Generic node to execute an arbitrary agent and returning messages"""
    try:
        result = await agent.ainvoke(state)

        if isinstance(result, dict) and "messages" in result:
            return {"messages": [AIMessage(content=result["messages"][-1].content, name=name)]}
        return {"messages": [AIMessage(content=str(result), name=name)]}
    except Exception as e:
        return {"messages": [AIMessage(content=f"An error occurred: {str(e)}", name=name)]}

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.managed import IsLastStep

class CustomState(TypedDict):
        url: str
        topics: list[str]
        age: str
        count: int
        messages: Annotated[list[BaseMessage], add_messages]
        is_last_step: IsLastStep

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver

prompt = hub.pull("content-finder")
# prompt = hub.pull("ih/ih-react-agent-executor")
# prompt = ChatPromptTemplate([
#     ("system", "You are a helpful assistant."),
#     ("user", "What is the weather like in {city}?"),
# ])
agent_executor = create_react_agent(model, tools, state_schema=CustomState, state_modifier=prompt, checkpointer=MemorySaver())

In [ ]:
input_values = {
    # "url": "https://techcommunity.microsoft.com/group/azure-tech-bites?messages.widget.messagelistfornodebyrecentactivitywidget-tab-main-group-hub-widgets-0=mostRecent",
    "url": "https://letsbuildazure.com/",
    "topics": ["Azure", "security", "networking", "AI", "kubernetes", "devops"],
    "age" : "2 weeks",
    "count": 5,
}
config = {"configurable": {"thread_id": "thread-1"}}
agent_executor.invoke(input_values, config=config)